# Capstone Project - The Battle of the Neighborhoods
## Places to eat at while in Australia.

The following lines of code import the required libraries for the notebook.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('\n')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

Foursquare API credentials are added into the notebook.

In [2]:
# The code was removed by Watson Studio for sharing.

Foursquare API Credentials successfully added to the notebook.


We now define a function that allows us to access various venue details for a given venue id and place it into a table if it has a rating. For this project we only require the rating.

In [3]:
def get_venue_details(venue_id):
         
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_rating=venue_data['rating']
        venue_details.append([venue_id,venue_name,venue_rating])
    except KeyError:
        pass
    
    column_names=['ID','Name','Rating']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

Each city will use the same set of code to collect its results.

#### Melbourne

The geopy library is used to get the latitude and longitude of a given street in each of the 5 cities that will allow us to get the exact point of each train station after a certain code is run.

In [4]:
address = 'Flinders St, VIC'

geolocator = Nominatim(user_agent="vic_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates successfully found!')

Coordinates successfully found!


A search query is used to find the train station within 500 metres of the given address. The search is limited to 10 results. \
The corresponding URL is then defined.

All URL's defined in this notebook are used in conjunction with the _GET_ request to collect the information from Foursquare API.

In [5]:
search_query = 'Train Station'
radius = 500
LIMIT = 10
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

Train Station .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=JBE44IHG4Y3MP003PZ5ADZCLN5K2LXHCC0YBPF01KIIXBUNL&client_secret=BI5HKYUDKM4GRI45Q0BZSH5FPQCDEMGBB5PCXXUJDLI1BZHQ&ll=-37.8179793,144.9671293&v=20200531&query=Train Station&radius=500&limit=10'

After using the _GET_ request, the information is then transformed into a pandas dataframe.

In [6]:
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,4b058754f964a520a38c22e3,Flinders St.,AU,Melbourne,Australia,at St. Kilda Rd.,55,"[Flinders St. (at St. Kilda Rd.), Melbourne VI...",NaN,-37.818261,144.966607,3000,VIC,Flinders Street Station,v-1590978564
1,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",False,4c510747991c20a177bd1d87,Swanston St,AU,Melbourne,Australia,at Flinders Street Station,38,"[Swanston St (at Flinders Street Station), Mel...","[{'label': 'display', 'lat': -37.8181263002096...",-37.818126,144.967523,3000,VIC,Flinders Station Bakehouse,v-1590978564
2,"[{'id': '4bf58dd8d48988d117951735', 'name': 'C...",False,4bfa185e5efe2d7fdb3b6d34,"Shop 4, Flinders Street Railway Station",AU,Melbourne,Australia,Flinders St.,43,"[Shop 4, Flinders Street Railway Station (Flin...","[{'label': 'display', 'lat': -37.8181564097619...",-37.818156,144.967565,3000,VIC,Sugar Station,v-1590978564
3,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",False,4fa1ba4fe4b0a22f80f19488,NaN,AU,Melbourne,Australia,NaN,28,"[Melbourne VIC, Australia]","[{'label': 'display', 'lat': -37.8177424469860...",-37.817742,144.967258,NaN,VIC,Flinders Street Station Pharmacy,v-1590978564
4,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",False,4c64f024c74c1b8d8fecb283,Flinders St,AU,Melbourne,Australia,St Kilda Rd,43,"[Flinders St (St Kilda Rd), Melbourne VIC 3000...","[{'label': 'display', 'lat': -37.8176495595111...",-37.817650,144.967389,3000,VIC,Flinders Street Station Steps,v-1590978564
5,"[{'id': '4bf58dd8d48988d148941735', 'name': 'D...",False,4ca4f1818a65bfb75f462b22,"Shop 2, Flinders Street Station, Cnr Flinders ...",AU,Melbourne,Australia,cnr Flinders and Swanston Sts,31,"[Shop 2, Flinders Street Station, Cnr Flinders...","[{'label': 'display', 'lat': -37.8181302363798...",-37.818130,144.967430,3000,VIC,Spanish Doughnuts,v-1590978564
6,"[{'id': '4bf58dd8d48988d12a951735', 'name': 'T...",False,58dad8a42980db73e94058fd,NaN,AU,Melbourne,Australia,NaN,44,"[Melbourne VIC 3000, Australia]","[{'label': 'display', 'lat': -37.818127, 'lng'...",-37.818127,144.966655,3000,VIC,Dandenong Train,v-1590978564
7,"[{'id': '4bf58dd8d48988d1fc931735', 'name': 'L...",False,4c133587a1010f47ad554a18,Flinders St.,AU,Melbourne,Australia,at Swanston St.,37,"[Flinders St. (at Swanston St.), Melbourne VIC...","[{'label': 'display', 'lat': -37.8176584603705...",-37.817658,144.966998,3000,VIC,Tram Stop 5 (70/75/Circle),v-1590978564
8,"[{'id': '4bf58dd8d48988d12a951735', 'name': 'T...",False,55e792d6498e79d4e6abbcfd,NaN,AU,NaN,Australia,NaN,55,[Australia],"[{'label': 'display', 'lat': -37.818259, 'lng'...",-37.818259,144.966603,NaN,NaN,Pakenham Train,v-1590978564
9,"[{'id': '4bf58dd8d48988d12a951735', 'name': 'T...",False,56578246498e9760a6be5920,NaN,AU,NaN,Australia,NaN,49,[Australia],"[{'label': 'display', 'lat': -37.818073, 'lng'...",-37.818073,144.966578,NaN,NaN,Epping Train,v-1590978564


The dataframe is then filtered to only include the required details.

In [7]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Flinders Street Station,Train Station,Flinders St.,AU,Melbourne,Australia,at St. Kilda Rd.,55,"[Flinders St. (at St. Kilda Rd.), Melbourne VI...",NaN,-37.818261,144.966607,3000,VIC,4b058754f964a520a38c22e3
1,Flinders Station Bakehouse,Bakery,Swanston St,AU,Melbourne,Australia,at Flinders Street Station,38,"[Swanston St (at Flinders Street Station), Mel...","[{'label': 'display', 'lat': -37.8181263002096...",-37.818126,144.967523,3000,VIC,4c510747991c20a177bd1d87
2,Sugar Station,Candy Store,"Shop 4, Flinders Street Railway Station",AU,Melbourne,Australia,Flinders St.,43,"[Shop 4, Flinders Street Railway Station (Flin...","[{'label': 'display', 'lat': -37.8181564097619...",-37.818156,144.967565,3000,VIC,4bfa185e5efe2d7fdb3b6d34
3,Flinders Street Station Pharmacy,Pharmacy,NaN,AU,Melbourne,Australia,NaN,28,"[Melbourne VIC, Australia]","[{'label': 'display', 'lat': -37.8177424469860...",-37.817742,144.967258,NaN,VIC,4fa1ba4fe4b0a22f80f19488
4,Flinders Street Station Steps,Plaza,Flinders St,AU,Melbourne,Australia,St Kilda Rd,43,"[Flinders St (St Kilda Rd), Melbourne VIC 3000...","[{'label': 'display', 'lat': -37.8176495595111...",-37.817650,144.967389,3000,VIC,4c64f024c74c1b8d8fecb283
5,Spanish Doughnuts,Donut Shop,"Shop 2, Flinders Street Station, Cnr Flinders ...",AU,Melbourne,Australia,cnr Flinders and Swanston Sts,31,"[Shop 2, Flinders Street Station, Cnr Flinders...","[{'label': 'display', 'lat': -37.8181302363798...",-37.818130,144.967430,3000,VIC,4ca4f1818a65bfb75f462b22
6,Dandenong Train,Train,NaN,AU,Melbourne,Australia,NaN,44,"[Melbourne VIC 3000, Australia]","[{'label': 'display', 'lat': -37.818127, 'lng'...",-37.818127,144.966655,3000,VIC,58dad8a42980db73e94058fd
7,Tram Stop 5 (70/75/Circle),Light Rail Station,Flinders St.,AU,Melbourne,Australia,at Swanston St.,37,"[Flinders St. (at Swanston St.), Melbourne VIC...","[{'label': 'display', 'lat': -37.8176584603705...",-37.817658,144.966998,3000,VIC,4c133587a1010f47ad554a18
8,Pakenham Train,Train,NaN,AU,NaN,Australia,NaN,55,[Australia],"[{'label': 'display', 'lat': -37.818259, 'lng'...",-37.818259,144.966603,NaN,NaN,55e792d6498e79d4e6abbcfd
9,Epping Train,Train,NaN,AU,NaN,Australia,NaN,49,[Australia],"[{'label': 'display', 'lat': -37.818073, 'lng'...",-37.818073,144.966578,NaN,NaN,56578246498e9760a6be5920


The following lines of code allow us to access the exact coordinates of the train station.

In [8]:
flindersst_df = pd.DataFrame(dataframe_filtered.head(1))
flindersst_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Flinders Street Station,Train Station,Flinders St.,AU,Melbourne,Australia,at St. Kilda Rd.,55,"[Flinders St. (at St. Kilda Rd.), Melbourne VI...",NaN,-37.818261,144.966607,3000,VIC,4b058754f964a520a38c22e3


In [9]:
flinlat = flindersst_df.iloc[0]['lat']
flinlng = flindersst_df.iloc[0]['lng']
print('Coordinates successfully loaded!')

Coordinates successfully loaded!


The search query is now used to find restaurants within 500 metres of the train station. The search is limited to 50 results.

The same process is followed to create a dataframe that lists the restaurants, their location details and their proximity to the train station.

In [10]:
search_query = 'Restaurant'
radius = 500
LIMIT = 50
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, flinlat, flinlng, VERSION, search_query, radius, LIMIT)
url

Restaurant .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=JBE44IHG4Y3MP003PZ5ADZCLN5K2LXHCC0YBPF01KIIXBUNL&client_secret=BI5HKYUDKM4GRI45Q0BZSH5FPQCDEMGBB5PCXXUJDLI1BZHQ&ll=-37.81826121222583,144.96660713480878&v=20200531&query=Restaurant&radius=500&limit=50'

In [11]:
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",False,5036122fe4b0853eeba6e6ec,100 St Kilda Rd,AU,Melbourne,Australia,NaN,177,"[100 St Kilda Rd, Melbourne VIC 3000, Australia]","[{'label': 'display', 'lat': -37.8195942086467...",-37.819594,144.967724,NaN,3000,VIC,Saké Restaurant and Bar,v-1590978567,NaN
1,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,4b058747f964a520928822e3,"River Terrace, Yarra Building, Federation Square",AU,Melbourne,Australia,NaN,205,"[River Terrace, Yarra Building, Federation Squ...","[{'label': 'display', 'lat': -37.8184170299779...",-37.818417,144.968939,NaN,3000,VIC,Feddish Cafe Bar & Restaurant,v-1590978567,NaN
2,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4b05874cf964a520f78922e3,Village Arcade,AU,Melbourne,Australia,189 Little Bourke St.,694,"[Village Arcade (189 Little Bourke St.), Melbo...","[{'label': 'display', 'lat': -37.8120274346454...",-37.812027,144.966900,NaN,3000,VIC,West Lake Restaurant,v-1590978567,NaN
3,"[{'id': '4bf58dd8d48988d169941735', 'name': 'A...",False,4c10bdba3ce120a1a37c081c,205 Collins St.,AU,Melbourne,Australia,at Swanston St.,291,"[205 Collins St. (at Swanston St.), Melbourne ...","[{'label': 'display', 'lat': -37.8156516894760...",-37.815652,144.966935,Melbourne CBD,3000,VIC,Allegro Restaurant,v-1590978567,NaN
4,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...",False,4b05874df964a5204c8a22e3,"Level 1, 139 Swanston St.",AU,Melbourne,Australia,NaN,436,"[Level 1, 139 Swanston St., Melbourne VIC 3000...","[{'label': 'display', 'lat': -37.8143516018564...",-37.814352,144.966167,NaN,3000,VIC,Gopal's Pure Vegetarian Restaurant,v-1590978567,48687685


In [12]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Saké Restaurant and Bar,Japanese Restaurant,100 St Kilda Rd,AU,Melbourne,Australia,NaN,177,"[100 St Kilda Rd, Melbourne VIC 3000, Australia]","[{'label': 'display', 'lat': -37.8195942086467...",-37.819594,144.967724,NaN,3000,VIC,5036122fe4b0853eeba6e6ec
1,Feddish Cafe Bar & Restaurant,Bar,"River Terrace, Yarra Building, Federation Square",AU,Melbourne,Australia,NaN,205,"[River Terrace, Yarra Building, Federation Squ...","[{'label': 'display', 'lat': -37.8184170299779...",-37.818417,144.968939,NaN,3000,VIC,4b058747f964a520928822e3
2,West Lake Restaurant,Chinese Restaurant,Village Arcade,AU,Melbourne,Australia,189 Little Bourke St.,694,"[Village Arcade (189 Little Bourke St.), Melbo...","[{'label': 'display', 'lat': -37.8120274346454...",-37.812027,144.966900,NaN,3000,VIC,4b05874cf964a520f78922e3
3,Allegro Restaurant,Australian Restaurant,205 Collins St.,AU,Melbourne,Australia,at Swanston St.,291,"[205 Collins St. (at Swanston St.), Melbourne ...","[{'label': 'display', 'lat': -37.8156516894760...",-37.815652,144.966935,Melbourne CBD,3000,VIC,4c10bdba3ce120a1a37c081c
4,Gopal's Pure Vegetarian Restaurant,Vegetarian / Vegan Restaurant,"Level 1, 139 Swanston St.",AU,Melbourne,Australia,NaN,436,"[Level 1, 139 Swanston St., Melbourne VIC 3000...","[{'label': 'display', 'lat': -37.8143516018564...",-37.814352,144.966167,NaN,3000,VIC,4b05874df964a5204c8a22e3
5,Alice's Thai Restaurant,Thai Restaurant,"Victoria Hotel, 215 Little Collins St.",AU,Melbourne,Australia,NaN,441,"[Victoria Hotel, 215 Little Collins St., Melbo...","[{'label': 'display', 'lat': -37.8142928206786...",-37.814293,144.966635,NaN,3000,VIC,4c0ae0bea1b32d7fdb2a9af0
6,Rainbow Buffet Restaurant,Asian Restaurant,2/206 Bourke St,AU,Melbourne,Australia,NaN,582,"[2/206 Bourke St, Melbourne VIC 3000, Australia]","[{'label': 'display', 'lat': -37.8130423856889...",-37.813042,144.967129,NaN,3000,VIC,4da81819a86e68edb7d4bc2a
7,Shanghai Dynasty Restaurant,Chinese Restaurant,"Level 1, 206 Bourke St",AU,Melbourne,Australia,NaN,588,"[Level 1, 206 Bourke St, Melbourne VIC 3000, A...","[{'label': 'display', 'lat': -37.8129912155587...",-37.812991,144.967075,NaN,3000,VIC,4d1417e2bb64224b6f3daa65
8,Hongyun Chinese Restaurant 鸿运面馆,Asian Restaurant,209-211 Bourke St,AU,Melbourne,Australia,NaN,582,"[209-211 Bourke St, Melbourne VIC 3000, Austra...","[{'label': 'display', 'lat': -37.8130642471239...",-37.813064,144.967344,NaN,3000,VIC,4c5e69ba2815c9289db8b767
9,Flora Indian Restaurant,Indian Restaurant,238A Flinders St.,AU,Melbourne,Australia,NaN,74,"[238A Flinders St., Melbourne VIC 3000, Austra...","[{'label': 'display', 'lat': -37.8175919440911...",-37.817592,144.966626,NaN,3000,VIC,4b307875f964a520c0f924e3


A dataframe is now created that contains the id's of the 50 restaurants.

In [13]:
venues_id = dataframe_filtered['id']
venues_id.head()

0    5036122fe4b0853eeba6e6ec
1    4b058747f964a520928822e3
2    4b05874cf964a520f78922e3
3    4c10bdba3ce120a1a37c081c
4    4b05874df964a5204c8a22e3
Name: id, dtype: object

The next step is to create a dataframe that shows the restaurant and its rating.

In [16]:
column_names=['ID','Name','Rating']
df_melb = pd.DataFrame(get_venue_details(venues_id[0]),columns=column_names)
df_melb

,ID,Name,Rating
0,5036122fe4b0853eeba6e6ec,Saké Restaurant and Bar,7.7


A list of series is passed to the append() to add in all the restaurants. If the restaurant doesn't have a rating it will not be included in the table.

In [17]:
df_melb = df_melb.append([get_venue_details(venues_id[1]), get_venue_details(venues_id[2]), get_venue_details(venues_id[3]),
                          get_venue_details(venues_id[4]), get_venue_details(venues_id[5]), get_venue_details(venues_id[6]),
                          get_venue_details(venues_id[7]), get_venue_details(venues_id[8]), get_venue_details(venues_id[9]),
                          get_venue_details(venues_id[10]), get_venue_details(venues_id[11]), get_venue_details(venues_id[12]),
                          get_venue_details(venues_id[13]), get_venue_details(venues_id[14]), get_venue_details(venues_id[15]),
                          get_venue_details(venues_id[16]), get_venue_details(venues_id[17]), get_venue_details(venues_id[18]),
                          get_venue_details(venues_id[19]), get_venue_details(venues_id[20]), get_venue_details(venues_id[21]),
                          get_venue_details(venues_id[22]), get_venue_details(venues_id[23]), get_venue_details(venues_id[24]),
                          get_venue_details(venues_id[25]), get_venue_details(venues_id[26]), get_venue_details(venues_id[27]),
                          get_venue_details(venues_id[28]), get_venue_details(venues_id[29]), get_venue_details(venues_id[30]),
                          get_venue_details(venues_id[31]), get_venue_details(venues_id[32]), get_venue_details(venues_id[33]),
                          get_venue_details(venues_id[34]), get_venue_details(venues_id[35]), get_venue_details(venues_id[36]),
                          get_venue_details(venues_id[37]), get_venue_details(venues_id[38]), get_venue_details(venues_id[39]),
                          get_venue_details(venues_id[40]), get_venue_details(venues_id[41]), get_venue_details(venues_id[42]),
                          get_venue_details(venues_id[43]), get_venue_details(venues_id[44]), get_venue_details(venues_id[45]),
                          get_venue_details(venues_id[46]), get_venue_details(venues_id[47]), get_venue_details(venues_id[48]),
                          get_venue_details(venues_id[49])], ignore_index = True)
df_melb

,ID,Name,Rating
0,5036122fe4b0853eeba6e6ec,Saké Restaurant and Bar,7.7
1,4b058747f964a520928822e3,Feddish Cafe Bar & Restaurant,5.2
2,4b05874cf964a520f78922e3,West Lake Restaurant,7.0
3,4c10bdba3ce120a1a37c081c,Allegro Restaurant,6.7
4,4b05874df964a5204c8a22e3,Gopal's Pure Vegetarian Restaurant,8.0
5,4c0ae0bea1b32d7fdb2a9af0,Alice's Thai Restaurant,6.9
6,4da81819a86e68edb7d4bc2a,Rainbow Buffet Restaurant,5.5
7,4d1417e2bb64224b6f3daa65,Shanghai Dynasty Restaurant,5.9
8,4b307875f964a520c0f924e3,Flora Indian Restaurant,6.1
9,54c2df00498e782c16f5d1b9,Platform 270 Restaurant & Bar,7.4


The describe function is now used to veiw different statistics about the rated restaurants.

In [18]:
df_melb.describe()

,Rating
count,20.000000
mean,6.685000
std,0.837462
min,5.200000
25%,6.100000
50%,6.650000
75%,7.400000
max,8.100000


<br/>

#### Sydney

In [19]:
address = 'Eddy Ave, NSW'

geolocator = Nominatim(user_agent="nsw_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates successfully found!')

Coordinates successfully found!


In [20]:
search_query = 'Train Station'
radius = 500
LIMIT = 10
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

Train Station .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=JBE44IHG4Y3MP003PZ5ADZCLN5K2LXHCC0YBPF01KIIXBUNL&client_secret=BI5HKYUDKM4GRI45Q0BZSH5FPQCDEMGBB5PCXXUJDLI1BZHQ&ll=-33.8816948,151.2061911&v=20200531&query=Train Station&radius=500&limit=10'

In [21]:
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,53c89c23498e585741c6716d,Eddy Ave,AU,Sydney,Australia,btw Chalmers St & Pitt St,258,"[Eddy Ave (btw Chalmers St & Pitt St), Sydney ...","[{'label': 'display', 'lat': -33.8839395981448...",-33.883940,151.206912,"Haymarket, Sydney, NSW",2000,NSW,Central Station,v-1590979739
1,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,4b058770f964a520249322e3,Eddy Ave.,AU,Surry Hills,Australia,btwn Chalmers & George St,267,"[Eddy Ave. (btwn Chalmers & George St), Surry ...","[{'label': 'display', 'lat': -33.8839216692723...",-33.883922,151.207265,Hayma,2000,NSW,Central Station (Northern Concourse),v-1590979739
2,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,4becf4ce91380f47ef509f18,Eddy Ave,AU,Sydney,Australia,NaN,164,"[Eddy Ave, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8828961857611...",-33.882896,151.207225,NaN,2000,NSW,Central Station (Eddy Avenue Forecourt),v-1590979739
3,"[{'id': '4bf58dd8d48988d12b951735', 'name': 'B...",False,4e7e7a4a49015dcee468fb51,486 Pitt St.,AU,Haymarket,Australia,NaN,91,"[486 Pitt St., Haymarket NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8821342480260...",-33.882134,151.205360,NaN,2000,NSW,Murrays Coaches - Central Station,v-1590979739
4,"[{'id': '4f4531504b9074f6e4fb0102', 'name': 'P...",False,4c64e633cdb6e21ef8db333e,Eddy Ave.,AU,Sydney,Australia,btwn Chalmers St. & George St.,131,"[Eddy Ave. (btwn Chalmers St. & George St.), S...","[{'label': 'display', 'lat': -33.8828729374123...",-33.882873,151.206089,NaN,2000,NSW,Platforms 2 & 3,v-1590979739
5,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4b59f6bff964a52031a428e3,Central Station,AU,Sydney,Australia,Eddy Ave.,185,"[Central Station (Eddy Ave.), Sydney NSW 2000,...","[{'label': 'display', 'lat': -33.8830868560518...",-33.883087,151.207301,NaN,2000,NSW,Train Crew Assignment Centre,v-1590979739
6,"[{'id': '4bf58dd8d48988d130951735', 'name': 'T...",False,4d4669674e5d3704046b0194,Eddy Ave.,AU,Sydney,Australia,NaN,103,"[Eddy Ave., Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8822803877707...",-33.882280,151.207066,NaN,2000,NSW,Taxi Rank - Central Station,v-1590979739
7,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,4b0b7143f964a520873123e3,Elizabeth St.,AU,Sydney,Australia,Liverpool St.,645,"[Elizabeth St. (Liverpool St.), Sydney NSW 200...","[{'label': 'display', 'lat': -33.8766996640586...",-33.876700,151.209732,NaN,2000,NSW,Museum Station (Concourse),v-1590979739
8,"[{'id': '4bf58dd8d48988d12b941735', 'name': 'C...",False,4ed885b0e5fa78b16bd94f4f,Eddy Ave,AU,Haymarket,Australia,NaN,67,"[Eddy Ave, Haymarket NSW, Australia]","[{'label': 'display', 'lat': -33.8822549385075...",-33.882255,151.205921,NaN,NaN,NSW,Central Station (Commuter Crime Unit),v-1590979739
9,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,4e7440dcd1643f93b1622f33,Eddy Ave,AU,Sydney,Australia,NaN,197,"[Eddy Ave, Sydney NSW, Australia]","[{'label': 'display', 'lat': -33.8832506567233...",-33.883251,151.207225,NaN,NaN,NSW,Central Station Bar,v-1590979739


In [22]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Central Station,Train Station,Eddy Ave,AU,Sydney,Australia,btw Chalmers St & Pitt St,258,"[Eddy Ave (btw Chalmers St & Pitt St), Sydney ...","[{'label': 'display', 'lat': -33.8839395981448...",-33.883940,151.206912,"Haymarket, Sydney, NSW",2000,NSW,53c89c23498e585741c6716d
1,Central Station (Northern Concourse),Train Station,Eddy Ave.,AU,Surry Hills,Australia,btwn Chalmers & George St,267,"[Eddy Ave. (btwn Chalmers & George St), Surry ...","[{'label': 'display', 'lat': -33.8839216692723...",-33.883922,151.207265,Hayma,2000,NSW,4b058770f964a520249322e3
2,Central Station (Eddy Avenue Forecourt),Train Station,Eddy Ave,AU,Sydney,Australia,NaN,164,"[Eddy Ave, Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8828961857611...",-33.882896,151.207225,NaN,2000,NSW,4becf4ce91380f47ef509f18
3,Murrays Coaches - Central Station,Bus Line,486 Pitt St.,AU,Haymarket,Australia,NaN,91,"[486 Pitt St., Haymarket NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8821342480260...",-33.882134,151.205360,NaN,2000,NSW,4e7e7a4a49015dcee468fb51
4,Platforms 2 & 3,Platform,Eddy Ave.,AU,Sydney,Australia,btwn Chalmers St. & George St.,131,"[Eddy Ave. (btwn Chalmers St. & George St.), S...","[{'label': 'display', 'lat': -33.8828729374123...",-33.882873,151.206089,NaN,2000,NSW,4c64e633cdb6e21ef8db333e
5,Train Crew Assignment Centre,Office,Central Station,AU,Sydney,Australia,Eddy Ave.,185,"[Central Station (Eddy Ave.), Sydney NSW 2000,...","[{'label': 'display', 'lat': -33.8830868560518...",-33.883087,151.207301,NaN,2000,NSW,4b59f6bff964a52031a428e3
6,Taxi Rank - Central Station,Taxi,Eddy Ave.,AU,Sydney,Australia,NaN,103,"[Eddy Ave., Sydney NSW 2000, Australia]","[{'label': 'display', 'lat': -33.8822803877707...",-33.882280,151.207066,NaN,2000,NSW,4d4669674e5d3704046b0194
7,Museum Station (Concourse),Train Station,Elizabeth St.,AU,Sydney,Australia,Liverpool St.,645,"[Elizabeth St. (Liverpool St.), Sydney NSW 200...","[{'label': 'display', 'lat': -33.8766996640586...",-33.876700,151.209732,NaN,2000,NSW,4b0b7143f964a520873123e3
8,Central Station (Commuter Crime Unit),Courthouse,Eddy Ave,AU,Haymarket,Australia,NaN,67,"[Eddy Ave, Haymarket NSW, Australia]","[{'label': 'display', 'lat': -33.8822549385075...",-33.882255,151.205921,NaN,NaN,NSW,4ed885b0e5fa78b16bd94f4f
9,Central Station Bar,Bar,Eddy Ave,AU,Sydney,Australia,NaN,197,"[Eddy Ave, Sydney NSW, Australia]","[{'label': 'display', 'lat': -33.8832506567233...",-33.883251,151.207225,NaN,NaN,NSW,4e7440dcd1643f93b1622f33


In [23]:
sydcnrl_df = pd.DataFrame(dataframe_filtered.head(1))
sydcnrl_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Central Station,Train Station,Eddy Ave,AU,Sydney,Australia,btw Chalmers St & Pitt St,258,"[Eddy Ave (btw Chalmers St & Pitt St), Sydney ...","[{'label': 'display', 'lat': -33.8839395981448...",-33.88394,151.206912,"Haymarket, Sydney, NSW",2000,NSW,53c89c23498e585741c6716d


In [24]:
sydcnrllat = sydcnrl_df.iloc[0]['lat']
sydcnrllng = sydcnrl_df.iloc[0]['lng']
print('Coordinates successfully loaded!')

Coordinates successfully loaded!


In [25]:
search_query = 'Restaurant'
radius = 500
LIMIT = 50
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, sydcnrllat, sydcnrllng, VERSION, search_query, radius, LIMIT)
url

Restaurant .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=JBE44IHG4Y3MP003PZ5ADZCLN5K2LXHCC0YBPF01KIIXBUNL&client_secret=BI5HKYUDKM4GRI45Q0BZSH5FPQCDEMGBB5PCXXUJDLI1BZHQ&ll=-33.88393959814486,151.2069116913331&v=20200531&query=Restaurant&radius=500&limit=50'

In [26]:
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4b058766f964a520729022e3,477 Pitt St,AU,Haymarket,Australia,at Hay St,419,"[477 Pitt St (at Hay St), Haymarket NSW 2000, ...","[{'label': 'display', 'lat': -33.8801834531476...",-33.880183,151.206489,Sydney,2000,NSW,Zilver Restaurant,v-1590979743
1,"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",False,4b058766f964a520939022e3,4-5/683-689 George St,AU,Haymarket,Australia,at Hay St,505,"[4-5/683-689 George St (at Hay St), Haymarket ...","[{'label': 'display', 'lat': -33.8797028017485...",-33.879703,151.204949,NaN,2000,NSW,Marigold Restaurant 富麗宮 (Marigold Restaurant),v-1590979743
2,"[{'id': '52e81612bcbc57f1066b79f8', 'name': 'P...",False,50e39502e4b0f21690a544e8,396 Elizabeth St,AU,Surry Hills,Australia,NaN,237,"[396 Elizabeth St, Surry Hills NSW 2010, Austr...","[{'label': 'display', 'lat': -33.8858159830388...",-33.885816,151.208135,NaN,2010,NSW,Himalaya Pakistani Indian Restaurant,v-1590979743
3,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",False,4da4250eb521224ba29b2bee,69 Regent St.,AU,Chippendale,Australia,NaN,485,"[69 Regent St., Chippendale NSW 2008, Australia]","[{'label': 'display', 'lat': -33.8860828491989...",-33.886083,151.202330,NaN,2008,NSW,Miyazaki Restaurant,v-1590979743
4,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",False,5da2c0191ebd700008d7ace6,421-429 Sussex St,AU,Haymarket,Australia,NaN,518,"[421-429 Sussex St, Haymarket NSW 2000, Austra...","[{'label': 'display', 'lat': -33.8799823725127...",-33.879982,151.203954,NaN,2000,NSW,East Ocean Seafood Restaurant 禧市,v-1590979743


In [27]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Zilver Restaurant,Chinese Restaurant,477 Pitt St,AU,Haymarket,Australia,at Hay St,419,"[477 Pitt St (at Hay St), Haymarket NSW 2000, ...","[{'label': 'display', 'lat': -33.8801834531476...",-33.880183,151.206489,Sydney,2000,NSW,4b058766f964a520729022e3
1,Marigold Restaurant 富麗宮 (Marigold Restaurant),Dim Sum Restaurant,4-5/683-689 George St,AU,Haymarket,Australia,at Hay St,505,"[4-5/683-689 George St (at Hay St), Haymarket ...","[{'label': 'display', 'lat': -33.8797028017485...",-33.879703,151.204949,NaN,2000,NSW,4b058766f964a520939022e3
2,Himalaya Pakistani Indian Restaurant,Pakistani Restaurant,396 Elizabeth St,AU,Surry Hills,Australia,NaN,237,"[396 Elizabeth St, Surry Hills NSW 2010, Austr...","[{'label': 'display', 'lat': -33.8858159830388...",-33.885816,151.208135,NaN,2010,NSW,50e39502e4b0f21690a544e8
3,Miyazaki Restaurant,Japanese Restaurant,69 Regent St.,AU,Chippendale,Australia,NaN,485,"[69 Regent St., Chippendale NSW 2008, Australia]","[{'label': 'display', 'lat': -33.8860828491989...",-33.886083,151.202330,NaN,2008,NSW,4da4250eb521224ba29b2bee
4,East Ocean Seafood Restaurant 禧市,Seafood Restaurant,421-429 Sussex St,AU,Haymarket,Australia,NaN,518,"[421-429 Sussex St, Haymarket NSW 2000, Austra...","[{'label': 'display', 'lat': -33.8799823725127...",-33.879982,151.203954,NaN,2000,NSW,5da2c0191ebd700008d7ace6
5,nel. restaurant,Australian Restaurant,75 Wentworth Ave.,AU,Sydney,Australia,at Elizabeth St.,555,"[75 Wentworth Ave. (at Elizabeth St.), Sydney ...","[{'label': 'display', 'lat': -33.8794426486323...",-33.879443,151.209507,NaN,2000,NSW,550a8849498ec17e9c53dd60
6,Harry's Singapore Chilli Crab Restaurant,Malay Restaurant,188 Elizabeth St,AU,Surry Hills,Australia,NaN,563,"[188 Elizabeth St, Surry Hills NSW 2010, Austr...","[{'label': 'display', 'lat': -33.8792668646522...",-33.879267,151.209250,NaN,2010,NSW,4b5d5e0cf964a520005b29e3
7,Daruma Japanese Restaurant,Japanese Restaurant,1/8 Quay St.,AU,Haymarket,Australia,Prince Centre,469,"[1/8 Quay St. (Prince Centre), Haymarket NSW 2...","[{'label': 'display', 'lat': -33.8811791689655...",-33.881179,151.203074,NaN,2000,NSW,4b3021d8f964a5209ff624e3
8,Santoor Indian Restaurant And Cafe,Indian Restaurant,4 Randle Ln.,AU,Surry Hills,Australia,NaN,120,"[4 Randle Ln., Surry Hills NSW 2010, Australia]","[{'label': 'display', 'lat': -33.8844610868633...",-33.884461,151.208056,NaN,2010,NSW,4bd9057211dcc92833c9f833
9,Wandee Thai Restaurant,Thai Restaurant,Shop1/60-62 Foveaux Street,AU,Surry Hills,Australia,Bellevue Street,389,"[Shop1/60-62 Foveaux Street (Bellevue Street),...","[{'label': 'display', 'lat': -33.8844920973975...",-33.884492,151.211071,NaN,2010,NSW,51650720e4b03b1860bbb7aa


In [28]:
venues_id = dataframe_filtered['id']
venues_id.head()

0    4b058766f964a520729022e3
1    4b058766f964a520939022e3
2    50e39502e4b0f21690a544e8
3    4da4250eb521224ba29b2bee
4    5da2c0191ebd700008d7ace6
Name: id, dtype: object

In [29]:
column_names=['ID','Name','Rating']
df_syd = pd.DataFrame(get_venue_details(venues_id[0]),columns=column_names)
df_syd

,ID,Name,Rating
0,4b058766f964a520729022e3,Zilver Restaurant,7.7


In [30]:
df_syd = df_syd.append([get_venue_details(venues_id[1]), get_venue_details(venues_id[2]), get_venue_details(venues_id[3]),
                        get_venue_details(venues_id[4]), get_venue_details(venues_id[5]), get_venue_details(venues_id[6]),
                        get_venue_details(venues_id[7]), get_venue_details(venues_id[8]), get_venue_details(venues_id[9]),
                        get_venue_details(venues_id[10]), get_venue_details(venues_id[11]), get_venue_details(venues_id[12]),
                        get_venue_details(venues_id[13]), get_venue_details(venues_id[14]), get_venue_details(venues_id[15]),
                        get_venue_details(venues_id[16]), get_venue_details(venues_id[17]), get_venue_details(venues_id[18]),
                        get_venue_details(venues_id[19]), get_venue_details(venues_id[20]), get_venue_details(venues_id[21]),
                        get_venue_details(venues_id[22]), get_venue_details(venues_id[23]), get_venue_details(venues_id[24]),
                        get_venue_details(venues_id[25]), get_venue_details(venues_id[26]), get_venue_details(venues_id[27]),
                        get_venue_details(venues_id[28]), get_venue_details(venues_id[29]), get_venue_details(venues_id[30]),
                        get_venue_details(venues_id[31]), get_venue_details(venues_id[32]), get_venue_details(venues_id[33]),
                        get_venue_details(venues_id[34]), get_venue_details(venues_id[35]), get_venue_details(venues_id[36]),
                        get_venue_details(venues_id[37]), get_venue_details(venues_id[38]), get_venue_details(venues_id[39]), 
                        get_venue_details(venues_id[40])], ignore_index=True)
df_syd

,ID,Name,Rating
0,4b058766f964a520729022e3,Zilver Restaurant,7.7
1,4b058766f964a520939022e3,Marigold Restaurant 富麗宮 (Marigold Restaurant),7.1
2,550a8849498ec17e9c53dd60,nel. restaurant,7.6
3,4b5d5e0cf964a520005b29e3,Harry's Singapore Chilli Crab Restaurant,6.7
4,4b3021d8f964a5209ff624e3,Daruma Japanese Restaurant,6.4
5,51650720e4b03b1860bbb7aa,Wandee Thai Restaurant,6.6
6,4da96e946a2303012ef3f90f,Four Elements Restaurant,6.2
7,4c836e4274d7b60c45b989d8,Capital Thai Restaurant,6.4
8,4b49c386f964a5205a7326e3,MillioRe Korean Fusion Restaurant,5.3
9,4c2c581877cfe21e9de4b5f1,Encasa Restaurant,7.1


In [31]:
df_syd.describe()

,Rating
count,22.000000
mean,7.009091
std,0.810590
min,5.300000
25%,6.425000
50%,7.000000
75%,7.575000
max,8.900000


<br/>

#### Brisbane

In [32]:
address = 'Roma St, QLD'

geolocator = Nominatim(user_agent="qld_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates successfully found!')

Coordinates successfully found!


In [33]:
search_query = 'Train Station'
radius = 500
LIMIT = 10
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

Train Station .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=JBE44IHG4Y3MP003PZ5ADZCLN5K2LXHCC0YBPF01KIIXBUNL&client_secret=BI5HKYUDKM4GRI45Q0BZSH5FPQCDEMGBB5PCXXUJDLI1BZHQ&ll=-27.464827550000003,153.01832108559603&v=20200531&query=Train Station&radius=500&limit=10'

In [34]:
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,4b04f0ecf964a520ed5622e3,171 Roma St.,AU,Brisbane,Australia,NaN,86,"[171 Roma St., Brisbane QLD 4000, Australia]","[{'label': 'display', 'lat': -27.4651179642124...",-27.465118,153.019134,4000,QLD,Roma Street Railway Station,v-1590979798
1,"[{'id': '4bf58dd8d48988d1fe931735', 'name': 'B...",False,4b061f93f964a520eae822e3,171 Roma St.,AU,Brisbane,Australia,NaN,177,"[171 Roma St., Brisbane QLD 4000, Australia]","[{'label': 'display', 'lat': -27.4660513948375...",-27.466051,153.019465,4000,QLD,Roma Street Busway Station,v-1590979798
2,"[{'id': '4f4531504b9074f6e4fb0102', 'name': 'P...",False,4bc3ece94cdfc9b68b1b9821,Roma St. Station,AU,Brisbane,Australia,NaN,82,"[Roma St. Station, Brisbane QLD 4000, Australia]","[{'label': 'display', 'lat': -27.4648307007621...",-27.464831,153.019153,4000,QLD,Platform 10,v-1590979798
3,"[{'id': '4bf58dd8d48988d1fe931735', 'name': 'B...",False,4b065f24f964a52040eb22e3,Adelaide St.,AU,Brisbane,Australia,at Albert St.,586,"[Adelaide St. (at Albert St.), Brisbane QLD 40...","[{'label': 'display', 'lat': -27.4678747439232...",-27.467875,153.023161,4000,QLD,King George Square Busway Station,v-1590979798
4,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4bb11adef964a520c67a3ce3,Roma Street Station,AU,Brisbane,Australia,NaN,81,"[Roma Street Station, Brisbane QLD 4000, Austr...","[{'label': 'display', 'lat': -27.4651773073559...",-27.465177,153.019044,4000,QLD,Zoom Cafe,v-1590979798
5,"[{'id': '4f4531504b9074f6e4fb0102', 'name': 'P...",False,4cbe9b047dc9a0938cfa33f5,Roma Street Station,AU,Brisbane,Australia,NaN,90,"[Roma Street Station, Brisbane QLD 4000, Austr...","[{'label': 'display', 'lat': -27.4652777174539...",-27.465278,153.019082,4000,QLD,Platform 5,v-1590979798
6,"[{'id': '4bf58dd8d48988d12a951735', 'name': 'T...",False,4e434aa9c65b8210945da785,Roma St. Station,AU,Brisbane,Australia,NaN,81,"[Roma St. Station, Brisbane QLD 4000, Australia]","[{'label': 'display', 'lat': -27.4648704541242...",-27.464870,153.019145,4000,QLD,Queensland Sunlander Traveltrain,v-1590979798
7,"[{'id': '4f4531504b9074f6e4fb0102', 'name': 'P...",False,4bd73cffe914a593a3ce50fa,Roma St. Station,AU,Brisbane,Australia,NaN,91,"[Roma St. Station, Brisbane QLD 4000, Australia]","[{'label': 'display', 'lat': -27.4653833484942...",-27.465383,153.019008,4000,QLD,Platform 4,v-1590979798
8,"[{'id': '4bf58dd8d48988d1e5931735', 'name': 'M...",False,5662942d498e1539065c8a10,NaN,AU,Brisbane,Australia,NaN,139,"[Brisbane QLD, Australia]","[{'label': 'display', 'lat': -27.4660175, 'lng...",-27.466017,153.018758,NaN,QLD,Destination Blues Train,v-1590979798
9,"[{'id': '4f4531504b9074f6e4fb0102', 'name': 'P...",False,4c5b31477735c9b695068972,Roma St. Station,AU,Brisbane,Australia,NaN,86,"[Roma St. Station, Brisbane QLD 4000, Australia]","[{'label': 'display', 'lat': -27.4650805666844...",-27.465081,153.019144,4000,QLD,Platform 7,v-1590979798


In [35]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Roma Street Railway Station,Train Station,171 Roma St.,AU,Brisbane,Australia,NaN,86,"[171 Roma St., Brisbane QLD 4000, Australia]","[{'label': 'display', 'lat': -27.4651179642124...",-27.465118,153.019134,4000,QLD,4b04f0ecf964a520ed5622e3
1,Roma Street Busway Station,Bus Station,171 Roma St.,AU,Brisbane,Australia,NaN,177,"[171 Roma St., Brisbane QLD 4000, Australia]","[{'label': 'display', 'lat': -27.4660513948375...",-27.466051,153.019465,4000,QLD,4b061f93f964a520eae822e3
2,Platform 10,Platform,Roma St. Station,AU,Brisbane,Australia,NaN,82,"[Roma St. Station, Brisbane QLD 4000, Australia]","[{'label': 'display', 'lat': -27.4648307007621...",-27.464831,153.019153,4000,QLD,4bc3ece94cdfc9b68b1b9821
3,King George Square Busway Station,Bus Station,Adelaide St.,AU,Brisbane,Australia,at Albert St.,586,"[Adelaide St. (at Albert St.), Brisbane QLD 40...","[{'label': 'display', 'lat': -27.4678747439232...",-27.467875,153.023161,4000,QLD,4b065f24f964a52040eb22e3
4,Zoom Cafe,Café,Roma Street Station,AU,Brisbane,Australia,NaN,81,"[Roma Street Station, Brisbane QLD 4000, Austr...","[{'label': 'display', 'lat': -27.4651773073559...",-27.465177,153.019044,4000,QLD,4bb11adef964a520c67a3ce3
5,Platform 5,Platform,Roma Street Station,AU,Brisbane,Australia,NaN,90,"[Roma Street Station, Brisbane QLD 4000, Austr...","[{'label': 'display', 'lat': -27.4652777174539...",-27.465278,153.019082,4000,QLD,4cbe9b047dc9a0938cfa33f5
6,Queensland Sunlander Traveltrain,Train,Roma St. Station,AU,Brisbane,Australia,NaN,81,"[Roma St. Station, Brisbane QLD 4000, Australia]","[{'label': 'display', 'lat': -27.4648704541242...",-27.464870,153.019145,4000,QLD,4e434aa9c65b8210945da785
7,Platform 4,Platform,Roma St. Station,AU,Brisbane,Australia,NaN,91,"[Roma St. Station, Brisbane QLD 4000, Australia]","[{'label': 'display', 'lat': -27.4653833484942...",-27.465383,153.019008,4000,QLD,4bd73cffe914a593a3ce50fa
8,Destination Blues Train,Music Venue,NaN,AU,Brisbane,Australia,NaN,139,"[Brisbane QLD, Australia]","[{'label': 'display', 'lat': -27.4660175, 'lng...",-27.466017,153.018758,NaN,QLD,5662942d498e1539065c8a10
9,Platform 7,Platform,Roma St. Station,AU,Brisbane,Australia,NaN,86,"[Roma St. Station, Brisbane QLD 4000, Australia]","[{'label': 'display', 'lat': -27.4650805666844...",-27.465081,153.019144,4000,QLD,4c5b31477735c9b695068972


In [36]:
romast_df = pd.DataFrame(dataframe_filtered.head(1))
romast_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Roma Street Railway Station,Train Station,171 Roma St.,AU,Brisbane,Australia,NaN,86,"[171 Roma St., Brisbane QLD 4000, Australia]","[{'label': 'display', 'lat': -27.4651179642124...",-27.465118,153.019134,4000,QLD,4b04f0ecf964a520ed5622e3


In [37]:
romastlat = romast_df.iloc[0]['lat']
romastlng = romast_df.iloc[0]['lng']
print('Coordinates successfully loaded!')

Coordinates successfully loaded!


In [38]:
search_query = 'Restaurant'
radius = 500
LIMIT = 50
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, romastlat, romastlng, VERSION, search_query, radius, LIMIT)
url

Restaurant .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=JBE44IHG4Y3MP003PZ5ADZCLN5K2LXHCC0YBPF01KIIXBUNL&client_secret=BI5HKYUDKM4GRI45Q0BZSH5FPQCDEMGBB5PCXXUJDLI1BZHQ&ll=-27.465117964212478,153.01913380622864&v=20200531&query=Restaurant&radius=500&limit=50'

In [39]:
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4b058739f964a520a88522e3,23 Leichhardt Street,AU,Spring Hill,Australia,Hotel Grand Chancellor,389,[23 Leichhardt Street (Hotel Grand Chancellor)...,"[{'label': 'display', 'lat': -27.46243, 'lng':...",-27.462430,153.021660,4000,QLD,Fresco's Restaurant,v-1590979801
1,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,4b058738f964a5204c8522e3,345 Wickham Terrace,AU,Brisbane,Australia,NaN,234,"[345 Wickham Terrace, Brisbane QLD 4000, Austr...","[{'label': 'display', 'lat': -27.46419, 'lng':...",-27.464190,153.021270,4000,QLD,Gazebo Restaurant and Bar,v-1590979801
2,[],False,559bf0d9498efd323d3fb21d,NaN,AU,NaN,Australia,NaN,397,[Australia],"[{'label': 'display', 'lat': -27.468671, 'lng'...",-27.468671,153.018780,NaN,NaN,Mundo Churrasco Restaurant and Bar,v-1590979801
3,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",False,4b6932a6f964a520809b2be3,500 Boundary St,AU,Spring Hill,Australia,NaN,581,"[500 Boundary St, Spring Hill QLD 4004, Austra...","[{'label': 'display', 'lat': -27.4604284281353...",-27.460428,153.021727,4004,QLD,Bishamon Japanese Restaurant,v-1590979801
4,"[{'id': '4bf58dd8d48988d1d5941735', 'name': 'H...",False,4c7cce2cad90224b1c208b0c,551 Wickham Terrace,AU,Spring Hill,Australia,NaN,467,"[551 Wickham Terrace, Spring Hill QLD 4000, Au...","[{'label': 'display', 'lat': -27.46092, 'lng':...",-27.460920,153.019110,4000,QLD,551 Restaurant & Bar,v-1590979801


In [40]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Fresco's Restaurant,Restaurant,23 Leichhardt Street,AU,Spring Hill,Australia,Hotel Grand Chancellor,389,[23 Leichhardt Street (Hotel Grand Chancellor)...,"[{'label': 'display', 'lat': -27.46243, 'lng':...",-27.462430,153.021660,4000,QLD,4b058739f964a520a88522e3
1,Gazebo Restaurant and Bar,Restaurant,345 Wickham Terrace,AU,Brisbane,Australia,NaN,234,"[345 Wickham Terrace, Brisbane QLD 4000, Austr...","[{'label': 'display', 'lat': -27.46419, 'lng':...",-27.464190,153.021270,4000,QLD,4b058738f964a5204c8522e3
2,Mundo Churrasco Restaurant and Bar,None,NaN,AU,NaN,Australia,NaN,397,[Australia],"[{'label': 'display', 'lat': -27.468671, 'lng'...",-27.468671,153.018780,NaN,NaN,559bf0d9498efd323d3fb21d
3,Bishamon Japanese Restaurant,Japanese Restaurant,500 Boundary St,AU,Spring Hill,Australia,NaN,581,"[500 Boundary St, Spring Hill QLD 4004, Austra...","[{'label': 'display', 'lat': -27.4604284281353...",-27.460428,153.021727,4004,QLD,4b6932a6f964a520809b2be3
4,551 Restaurant & Bar,Hotel Bar,551 Wickham Terrace,AU,Spring Hill,Australia,NaN,467,"[551 Wickham Terrace, Spring Hill QLD 4000, Au...","[{'label': 'display', 'lat': -27.46092, 'lng':...",-27.460920,153.019110,4000,QLD,4c7cce2cad90224b1c208b0c
5,Baan Thai Restaurant,Thai Restaurant,"Shop 1, 448 Boundary St.",AU,Spring Hill,Australia,NaN,608,"[Shop 1, 448 Boundary St., Spring Hill QLD 400...","[{'label': 'display', 'lat': -27.4605737204708...",-27.460574,153.022562,4000,QLD,4b45b87cf964a5208b0f26e3
6,Domani's Restaurant,Indian Restaurant,Corner Woodroffe and Tedder Avenue,AU,Gold Coast,Australia,NaN,560,"[Corner Woodroffe and Tedder Avenue, Gold Coas...","[{'label': 'display', 'lat': -27.4601051192603...",-27.460105,153.018630,4217,QLD,4b058739f964a520b88522e3
7,Govinda's,Vegetarian / Vegan Restaurant,358 George St,AU,Brisbane,Australia,NaN,473,"[358 George St, Brisbane QLD 4000, Australia]","[{'label': 'display', 'lat': -27.468844, 'lng'...",-27.468844,153.021436,4000,QLD,57e0a7eacd109d5981a1655c


In [41]:
venues_id = dataframe_filtered['id']
venues_id.head()

0    4b058739f964a520a88522e3
1    4b058738f964a5204c8522e3
2    559bf0d9498efd323d3fb21d
3    4b6932a6f964a520809b2be3
4    4c7cce2cad90224b1c208b0c
Name: id, dtype: object

In [42]:
column_names=['ID','Name','Rating']
df_bris = pd.DataFrame(get_venue_details(venues_id[0]),columns=column_names)
df_bris

,ID,Name,Rating


In [43]:
# Pass a list of series to the append() to add multiple rows
df_bris = df_bris.append([get_venue_details(venues_id[1]), get_venue_details(venues_id[2]), get_venue_details(venues_id[3]),
                          get_venue_details(venues_id[4]), get_venue_details(venues_id[5]), get_venue_details(venues_id[6]),
                          get_venue_details(venues_id[7])], ignore_index=True)
df_bris

,ID,Name,Rating
0,4b6932a6f964a520809b2be3,Bishamon Japanese Restaurant,8.4


<br/>

#### Perth

In [44]:
address = 'Perth, WA'

geolocator = Nominatim(user_agent="wa_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates successfully found!')

Coordinates successfully found!


In [45]:
search_query = 'Train Station'
radius = 500
LIMIT = 10
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

Train Station .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=JBE44IHG4Y3MP003PZ5ADZCLN5K2LXHCC0YBPF01KIIXBUNL&client_secret=BI5HKYUDKM4GRI45Q0BZSH5FPQCDEMGBB5PCXXUJDLI1BZHQ&ll=-31.9527121,115.8604796&v=20200531&query=Train Station&radius=500&limit=10'

In [46]:
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,4bac5619f964a520cef03ae3,Moore St,AU,Perth,Australia,Nash St,566,"[Moore St (Nash St), Perth WA 6000, Australia]","[{'label': 'display', 'lat': -31.9518163114900...",-31.951816,115.866387,Perth CBD,6000,WA,McIver Train Station,v-1590979934
1,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",False,4d44c7f7e198721e30a1bf8b,Wellington st Bus Depot concourse,AU,Perth,Australia,NaN,272,"[Wellington st Bus Depot concourse, Perth WA, ...","[{'label': 'display', 'lat': -31.951329, 'lng'...",-31.951329,115.858105,NaN,NaN,WA,Train Station News,v-1590979934
2,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,4b3c5acaf964a520008425e3,Wellington St,AU,Perth,Australia,NaN,150,"[Wellington St, Perth WA 6000, Australia]","[{'label': 'display', 'lat': -31.9513611430120...",-31.951361,115.860379,NaN,6000,WA,Perth Station,v-1590979934
3,"[{'id': '4bf58dd8d48988d1fd931735', 'name': 'M...",False,4b25ff46f964a520eb7724e3,William St.,AU,Perth,Australia,at Murray St,240,"[William St. (at Murray St), Perth WA 6000, Au...","[{'label': 'display', 'lat': -31.9520438948832...",-31.952044,115.858061,NaN,6000,WA,Perth Underground Station,v-1590979934
4,"[{'id': '56aa371be4b08b9a8d573517', 'name': 'B...",False,5d00ca74b5cb76002cffbcf7,3/271 Berkshire Road,AU,FORRESTFIELD,Australia,NaN,11,"[3/271 Berkshire Road, Forrestfield WA 6058, A...","[{'label': 'display', 'lat': -31.9527448481919...",-31.952745,115.860593,NaN,6058,WA,Nara Training and Assessing,v-1590979934
5,"[{'id': '4bf58dd8d48988d123951735', 'name': 'S...",False,4d1187f190d0a143b49e9e09,"Shop 3, 148 Murray St",AU,Perth,Australia,Barrack St.,79,"[Shop 3, 148 Murray St (Barrack St.), Perth WA...","[{'label': 'display', 'lat': -31.9533510430894...",-31.953351,115.860844,NaN,6000,WA,Tobacco Station,v-1590979934
6,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",False,4bf4bafb98ac0f47356664a8,Perth Train Station,AU,Perth,Australia,Wellington St,126,"[Perth Train Station (Wellington St), Perth WA...","[{'label': 'display', 'lat': -31.9515794494913...",-31.951579,115.860434,NaN,6000,WA,Priceline Pharmacy,v-1590979934
7,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4f74f4f1e4b0e09b8a7ab5f6,"Level 3, 166 Murray St",AU,Perth,Australia,NaN,32,"[Level 3, 166 Murray St, Perth WA 6000, Austra...","[{'label': 'display', 'lat': -31.9529897286726...",-31.952990,115.860601,NaN,6000,WA,Bhp Billiton Training Centre,v-1590979934
8,"[{'id': '4f4531504b9074f6e4fb0102', 'name': 'P...",False,4f094d1be4b0882d24064726,Perth Train Station,AU,Perth,Australia,NaN,163,"[Perth Train Station, Perth WA, Australia]","[{'label': 'display', 'lat': -31.9512899042372...",-31.951290,115.860038,NaN,NaN,WA,Platform 5,v-1590979934
9,"[{'id': '4f4531504b9074f6e4fb0102', 'name': 'P...",False,4bf8df845317a5930d8b007f,Perth Train Station,AU,Perth,Australia,btwn Beaufort St & William St,175,[Perth Train Station (btwn Beaufort St & Willi...,"[{'label': 'display', 'lat': -31.9511701931708...",-31.951170,115.860109,NaN,6000,WA,Platform 7,v-1590979934


In [47]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,McIver Train Station,Train Station,Moore St,AU,Perth,Australia,Nash St,566,"[Moore St (Nash St), Perth WA 6000, Australia]","[{'label': 'display', 'lat': -31.9518163114900...",-31.951816,115.866387,Perth CBD,6000,WA,4bac5619f964a520cef03ae3
1,Train Station News,Miscellaneous Shop,Wellington st Bus Depot concourse,AU,Perth,Australia,NaN,272,"[Wellington st Bus Depot concourse, Perth WA, ...","[{'label': 'display', 'lat': -31.951329, 'lng'...",-31.951329,115.858105,NaN,NaN,WA,4d44c7f7e198721e30a1bf8b
2,Perth Station,Train Station,Wellington St,AU,Perth,Australia,NaN,150,"[Wellington St, Perth WA 6000, Australia]","[{'label': 'display', 'lat': -31.9513611430120...",-31.951361,115.860379,NaN,6000,WA,4b3c5acaf964a520008425e3
3,Perth Underground Station,Metro Station,William St.,AU,Perth,Australia,at Murray St,240,"[William St. (at Murray St), Perth WA 6000, Au...","[{'label': 'display', 'lat': -31.9520438948832...",-31.952044,115.858061,NaN,6000,WA,4b25ff46f964a520eb7724e3
4,Nara Training and Assessing,Business Center,3/271 Berkshire Road,AU,FORRESTFIELD,Australia,NaN,11,"[3/271 Berkshire Road, Forrestfield WA 6058, A...","[{'label': 'display', 'lat': -31.9527448481919...",-31.952745,115.860593,NaN,6058,WA,5d00ca74b5cb76002cffbcf7
5,Tobacco Station,Smoke Shop,"Shop 3, 148 Murray St",AU,Perth,Australia,Barrack St.,79,"[Shop 3, 148 Murray St (Barrack St.), Perth WA...","[{'label': 'display', 'lat': -31.9533510430894...",-31.953351,115.860844,NaN,6000,WA,4d1187f190d0a143b49e9e09
6,Priceline Pharmacy,Pharmacy,Perth Train Station,AU,Perth,Australia,Wellington St,126,"[Perth Train Station (Wellington St), Perth WA...","[{'label': 'display', 'lat': -31.9515794494913...",-31.951579,115.860434,NaN,6000,WA,4bf4bafb98ac0f47356664a8
7,Bhp Billiton Training Centre,Office,"Level 3, 166 Murray St",AU,Perth,Australia,NaN,32,"[Level 3, 166 Murray St, Perth WA 6000, Austra...","[{'label': 'display', 'lat': -31.9529897286726...",-31.952990,115.860601,NaN,6000,WA,4f74f4f1e4b0e09b8a7ab5f6
8,Platform 5,Platform,Perth Train Station,AU,Perth,Australia,NaN,163,"[Perth Train Station, Perth WA, Australia]","[{'label': 'display', 'lat': -31.9512899042372...",-31.951290,115.860038,NaN,NaN,WA,4f094d1be4b0882d24064726
9,Platform 7,Platform,Perth Train Station,AU,Perth,Australia,btwn Beaufort St & William St,175,[Perth Train Station (btwn Beaufort St & Willi...,"[{'label': 'display', 'lat': -31.9511701931708...",-31.951170,115.860109,NaN,6000,WA,4bf8df845317a5930d8b007f


In [48]:
perst_df = pd.DataFrame(dataframe_filtered.iloc[[2]])
perst_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
2,Perth Station,Train Station,Wellington St,AU,Perth,Australia,NaN,150,"[Wellington St, Perth WA 6000, Australia]","[{'label': 'display', 'lat': -31.9513611430120...",-31.951361,115.860379,NaN,6000,WA,4b3c5acaf964a520008425e3


In [49]:
perstlat = perst_df.iloc[0]['lat']
perstlng = perst_df.iloc[0]['lng']
print('Coordinates successfully loaded!')

Coordinates successfully loaded!


In [50]:
search_query = 'Restaurant'
radius = 500
LIMIT = 50
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, perstlat, perstlng, VERSION, search_query, radius, LIMIT)
url

Restaurant .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=JBE44IHG4Y3MP003PZ5ADZCLN5K2LXHCC0YBPF01KIIXBUNL&client_secret=BI5HKYUDKM4GRI45Q0BZSH5FPQCDEMGBB5PCXXUJDLI1BZHQ&ll=-31.95136114301201,115.86037874221802&v=20200531&query=Restaurant&radius=500&limit=50'

In [51]:
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '52af3a7c3cf9994f4e043bed', 'name': 'C...",False,55ed79a3498e6eaf2fbcebde,20 Roe Sreet,AU,Northbridge,Australia,NaN,288,"[20 Roe Sreet, Northbridge WA, Australia]","[{'label': 'display', 'lat': -31.9496528483201...",-31.949653,115.858080,NaN,NaN,WA,Canton Bay Chinese Restaurant Northbridge 喜粵海鮮酒家,v-1590979938
1,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4b61256cf964a5201b0b2ae3,66 Francis St,AU,Northbridge,Australia,nr Lake St,462,"[66 Francis St (nr Lake St), Northbridge WA 60...","[{'label': 'display', 'lat': -31.9474889923021...",-31.947489,115.858610,NaN,6003,WA,Dragon Palace & Karaoke Restaurant,v-1590979938
2,"[{'id': '4bf58dd8d48988d1c6941735', 'name': 'S...",False,4c2c5bd38abca5938c940120,97 Murray St,AU,Perth,Australia,cnr Pier St,390,"[97 Murray St (cnr Pier St), Perth WA 6000, Au...","[{'label': 'display', 'lat': -31.9542861800683...",-31.954286,115.862652,NaN,6000,WA,Miss Maud Smorgasbord Restaurant,v-1590979938
3,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",False,4b74d676f964a520adf42de3,568 Hay St,AU,Perth,Australia,NaN,411,"[568 Hay St, Perth WA 6000, Australia]","[{'label': 'display', 'lat': -31.9549600047376...",-31.954960,115.861385,NaN,6000,WA,Restaurant Jun 純,v-1590979938
4,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4da98cd6a86e771ea70d1431,NaN,AU,Perth,Australia,NaN,409,"[Perth WA, Australia]","[{'label': 'display', 'lat': -31.9483931476421...",-31.948393,115.857810,NaN,NaN,WA,Tea Cafe Hong Kong Style BBQ Restaurant 茶餐廳,v-1590979938


In [52]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Canton Bay Chinese Restaurant Northbridge 喜粵海鮮酒家,Cantonese Restaurant,20 Roe Sreet,AU,Northbridge,Australia,NaN,288,"[20 Roe Sreet, Northbridge WA, Australia]","[{'label': 'display', 'lat': -31.9496528483201...",-31.949653,115.858080,NaN,NaN,WA,55ed79a3498e6eaf2fbcebde
1,Dragon Palace & Karaoke Restaurant,Chinese Restaurant,66 Francis St,AU,Northbridge,Australia,nr Lake St,462,"[66 Francis St (nr Lake St), Northbridge WA 60...","[{'label': 'display', 'lat': -31.9474889923021...",-31.947489,115.858610,NaN,6003,WA,4b61256cf964a5201b0b2ae3
2,Miss Maud Smorgasbord Restaurant,Scandinavian Restaurant,97 Murray St,AU,Perth,Australia,cnr Pier St,390,"[97 Murray St (cnr Pier St), Perth WA 6000, Au...","[{'label': 'display', 'lat': -31.9542861800683...",-31.954286,115.862652,NaN,6000,WA,4c2c5bd38abca5938c940120
3,Restaurant Jun 純,Japanese Restaurant,568 Hay St,AU,Perth,Australia,NaN,411,"[568 Hay St, Perth WA 6000, Australia]","[{'label': 'display', 'lat': -31.9549600047376...",-31.954960,115.861385,NaN,6000,WA,4b74d676f964a520adf42de3
4,Tea Cafe Hong Kong Style BBQ Restaurant 茶餐廳,Chinese Restaurant,NaN,AU,Perth,Australia,NaN,409,"[Perth WA, Australia]","[{'label': 'display', 'lat': -31.9483931476421...",-31.948393,115.857810,NaN,NaN,WA,4da98cd6a86e771ea70d1431
5,The Bonsai Restaurant Cafe & Lounge,Japanese Restaurant,30 Roe St,AU,Northbridge,Australia,NaN,326,"[30 Roe St, Northbridge WA 6003, Australia]","[{'label': 'display', 'lat': -31.9496463724188...",-31.949646,115.857576,NaN,6003,WA,4b5ae606f964a52065d928e3
6,Singapore Restaurant,Asian Restaurant,66 Roe St.,AU,Northbridge,Australia,btwn Lake St. And William St.,388,"[66 Roe St. (btwn Lake St. And William St.), N...","[{'label': 'display', 'lat': -31.9490709464152...",-31.949071,115.857269,NaN,6003,WA,4b9a2124f964a5204da035e3
7,Dejavu Tapas Restaurant,Tapas Restaurant,2/310 William St,AU,Northbridge,Australia,NaN,516,"[2/310 William St, Northbridge WA 6000, Austra...","[{'label': 'display', 'lat': -31.946753, 'lng'...",-31.946753,115.861046,Perth CBD,6000,WA,58746ac8ea1c0d5d161dd31f
8,City Garden Chinese Restaurant,Chinese Restaurant,11 Chungwah Ln,AU,Northbridge,Australia,Roe St,403,"[11 Chungwah Ln (Roe St), Northbridge WA 6003,...","[{'label': 'display', 'lat': -31.9489840705918...",-31.948984,115.857147,NaN,6003,WA,4b6c411ef964a520e82b2ce3
9,Ganges Indian Restaurant,Indian Restaurant,"3/200 Mirrabooka Ave, Alexander Heights 6064",AU,Perth,Australia,NaN,14,"[3/200 Mirrabooka Ave, Alexander Heights 6064,...","[{'label': 'display', 'lat': -31.9514885904131...",-31.951489,115.860430,NaN,6064,WA,5c10d7e4c0cacb002c6c60a9


In [53]:
venues_id = dataframe_filtered['id']
venues_id.head()

0    55ed79a3498e6eaf2fbcebde
1    4b61256cf964a5201b0b2ae3
2    4c2c5bd38abca5938c940120
3    4b74d676f964a520adf42de3
4    4da98cd6a86e771ea70d1431
Name: id, dtype: object

In [54]:
column_names=['ID','Name','Rating']
df_per = pd.DataFrame(get_venue_details(venues_id[0]),columns=column_names)
df_per

,ID,Name,Rating
0,55ed79a3498e6eaf2fbcebde,Canton Bay Chinese Restaurant Northbridge 喜粵海鮮酒家,7.3


In [55]:
df_per = df_per.append([get_venue_details(venues_id[1]), get_venue_details(venues_id[2]), get_venue_details(venues_id[3]),
                        get_venue_details(venues_id[4]), get_venue_details(venues_id[5]), get_venue_details(venues_id[6]),
                        get_venue_details(venues_id[7]), get_venue_details(venues_id[8]), get_venue_details(venues_id[9]),
                        get_venue_details(venues_id[10]), get_venue_details(venues_id[11]), get_venue_details(venues_id[12]),
                        get_venue_details(venues_id[13]), get_venue_details(venues_id[14]), get_venue_details(venues_id[15]),
                        get_venue_details(venues_id[16]), get_venue_details(venues_id[17]), get_venue_details(venues_id[18]),
                        get_venue_details(venues_id[19]), get_venue_details(venues_id[20]), get_venue_details(venues_id[21]),
                        get_venue_details(venues_id[22]), get_venue_details(venues_id[23]), get_venue_details(venues_id[24]),
                        get_venue_details(venues_id[25]), get_venue_details(venues_id[26]), get_venue_details(venues_id[27]),
                        get_venue_details(venues_id[28]), get_venue_details(venues_id[29]), get_venue_details(venues_id[30]),
                        get_venue_details(venues_id[31]), get_venue_details(venues_id[32]), get_venue_details(venues_id[33]),
                        get_venue_details(venues_id[34]), get_venue_details(venues_id[35]), get_venue_details(venues_id[36]),
                        get_venue_details(venues_id[37]), get_venue_details(venues_id[38]), get_venue_details(venues_id[39]),
                        get_venue_details(venues_id[40]), get_venue_details(venues_id[41]), get_venue_details(venues_id[42]),
                        get_venue_details(venues_id[43]), get_venue_details(venues_id[44]), get_venue_details(venues_id[45]),
                        get_venue_details(venues_id[46]), get_venue_details(venues_id[47]), get_venue_details(venues_id[48]),
                        get_venue_details(venues_id[49])], ignore_index=True)
df_per

,ID,Name,Rating
0,55ed79a3498e6eaf2fbcebde,Canton Bay Chinese Restaurant Northbridge 喜粵海鮮酒家,7.3
1,4b61256cf964a5201b0b2ae3,Dragon Palace & Karaoke Restaurant,7.1
2,4c2c5bd38abca5938c940120,Miss Maud Smorgasbord Restaurant,7.4
3,4b74d676f964a520adf42de3,Restaurant Jun 純,7.7
4,4b5ae606f964a52065d928e3,The Bonsai Restaurant Cafe & Lounge,8.0
5,4b9a2124f964a5204da035e3,Singapore Restaurant,7.7
6,4b6c411ef964a520e82b2ce3,City Garden Chinese Restaurant,7.0
7,4b8e4464f964a520691d33e3,Uncle Billy's Chinese Restaurant,6.9
8,4c6f5cdf9c6d6dcb0990cf7a,Poppo Korean & Japanese Restaurant,7.7
9,5349fd1d498ed98253ec6a6c,Fortune Five Chinese Restaurant,7.0


In [56]:
df_per.describe()

,Rating
count,22.000000
mean,6.968182
std,0.681306
min,5.600000
25%,6.600000
50%,7.100000
75%,7.400000
max,8.000000


<br/>

#### Adelaide

In [57]:
address = '125 North Terrace, SA'

geolocator = Nominatim(user_agent="sa_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates successfully found!')

Coordinates successfully found!


In [58]:
search_query = 'Train Station'
radius = 500
LIMIT = 10
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

Train Station .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=JBE44IHG4Y3MP003PZ5ADZCLN5K2LXHCC0YBPF01KIIXBUNL&client_secret=BI5HKYUDKM4GRI45Q0BZSH5FPQCDEMGBB5PCXXUJDLI1BZHQ&ll=-34.921406,138.6017329&v=20200531&query=Train Station&radius=500&limit=10'

In [59]:
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",False,4b440416f964a5203bf025e3,138 North Terrace,AU,Adelaide,Australia,NaN,493,"[138 North Terrace, Adelaide SA 5000, Australia]","[{'label': 'display', 'lat': -34.9208454200377...",-34.920845,138.596370,5000,SA,Adelaide Railway Station,v-1590980006
1,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",False,4ba057adf964a520696937e3,82 King William St,AU,Adelaide,Australia,cnr Grenfell St,371,"[82 King William St (cnr Grenfell St), Adelaid...","[{'label': 'display', 'lat': -34.9243761979758...",-34.924376,138.599883,5000,SA,Sushi Train,v-1590980006
2,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",False,4d8704f402eb5481af8062f5,38 Hindley Street,AU,Adelaide,Australia,NaN,381,"[38 Hindley Street, Adelaide SA 5000, Australia]","[{'label': 'display', 'lat': -34.92285, 'lng':...",-34.922850,138.597940,5000,SA,Go-in Hotpot Train,v-1590980006
3,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4b53bf48f964a52022aa27e3,52 Hindley St,AU,Adelaide,Australia,NaN,548,"[52 Hindley St, Adelaide SA, Australia]","[{'label': 'display', 'lat': -34.9215274190501...",-34.921527,138.595722,NaN,SA,Station Arcade,v-1590980006
4,"[{'id': '52f2ab2ebcbc57f1066b8b51', 'name': 'T...",False,4ba9dba6f964a520e63a3ae3,North Terrace,AU,Adelaide,Australia,NaN,395,"[North Terrace, Adelaide SA 5000, Australia]","[{'label': 'display', 'lat': -34.9216963027830...",-34.921696,138.597415,5000,SA,Adelaide Station Tram Stop,v-1590980006
5,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",False,4be50575bcef2d7ff56c03e5,Rundle Mall,AU,Adelaide,Australia,Regent Arcade,290,"[Rundle Mall (Regent Arcade), Adelaide SA 5000...","[{'label': 'display', 'lat': -34.9235431696448...",-34.923543,138.603566,5000,SA,Michiru Sushi Train,v-1590980006
6,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",False,4e7a774ad16401fc628cda19,38 Gawler Place,AU,Adelaide,Australia,NaN,141,"[38 Gawler Place, Adelaide SA, Australia]","[{'label': 'display', 'lat': -34.9226570129394...",-34.922657,138.601990,NaN,SA,Express Training Connections,v-1590980006
7,"[{'id': '4f4531504b9074f6e4fb0102', 'name': 'P...",False,4d8ad7dad85f3704dc9707dc,Adelaide Railway Station,AU,Adelaide,Australia,NaN,478,"[Adelaide Railway Station, Adelaide SA 5000, A...","[{'label': 'display', 'lat': -34.9208691484817...",-34.920869,138.596527,5000,SA,Platform 6,v-1590980006
8,"[{'id': '4f4531504b9074f6e4fb0102', 'name': 'P...",False,4df19e3ed4c01ff6b2e47352,Adelaide Train Station,AU,Adelaide,Australia,NaN,462,"[Adelaide Train Station, Adelaide SA 5000, Aus...","[{'label': 'display', 'lat': -34.9212320915168...",-34.921232,138.596671,5000,SA,Platform 5,v-1590980006
9,"[{'id': '4f4531504b9074f6e4fb0102', 'name': 'P...",False,4cfc991ac51fa1cdfac3ec2b,Adelaide Railway Station,AU,Adelaide,Australia,NaN,495,"[Adelaide Railway Station, Adelaide SA 5000, A...","[{'label': 'display', 'lat': -34.9207726666211...",-34.920773,138.596359,5000,SA,Platform 9,v-1590980006


In [60]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Adelaide Railway Station,Train Station,138 North Terrace,AU,Adelaide,Australia,NaN,493,"[138 North Terrace, Adelaide SA 5000, Australia]","[{'label': 'display', 'lat': -34.9208454200377...",-34.920845,138.596370,5000,SA,4b440416f964a5203bf025e3
1,Sushi Train,Sushi Restaurant,82 King William St,AU,Adelaide,Australia,cnr Grenfell St,371,"[82 King William St (cnr Grenfell St), Adelaid...","[{'label': 'display', 'lat': -34.9243761979758...",-34.924376,138.599883,5000,SA,4ba057adf964a520696937e3
2,Go-in Hotpot Train,Asian Restaurant,38 Hindley Street,AU,Adelaide,Australia,NaN,381,"[38 Hindley Street, Adelaide SA 5000, Australia]","[{'label': 'display', 'lat': -34.92285, 'lng':...",-34.922850,138.597940,5000,SA,4d8704f402eb5481af8062f5
3,Station Arcade,Shopping Mall,52 Hindley St,AU,Adelaide,Australia,NaN,548,"[52 Hindley St, Adelaide SA, Australia]","[{'label': 'display', 'lat': -34.9215274190501...",-34.921527,138.595722,NaN,SA,4b53bf48f964a52022aa27e3
4,Adelaide Station Tram Stop,Tram Station,North Terrace,AU,Adelaide,Australia,NaN,395,"[North Terrace, Adelaide SA 5000, Australia]","[{'label': 'display', 'lat': -34.9216963027830...",-34.921696,138.597415,5000,SA,4ba9dba6f964a520e63a3ae3
5,Michiru Sushi Train,Japanese Restaurant,Rundle Mall,AU,Adelaide,Australia,Regent Arcade,290,"[Rundle Mall (Regent Arcade), Adelaide SA 5000...","[{'label': 'display', 'lat': -34.9235431696448...",-34.923543,138.603566,5000,SA,4be50575bcef2d7ff56c03e5
6,Express Training Connections,Building,38 Gawler Place,AU,Adelaide,Australia,NaN,141,"[38 Gawler Place, Adelaide SA, Australia]","[{'label': 'display', 'lat': -34.9226570129394...",-34.922657,138.601990,NaN,SA,4e7a774ad16401fc628cda19
7,Platform 6,Platform,Adelaide Railway Station,AU,Adelaide,Australia,NaN,478,"[Adelaide Railway Station, Adelaide SA 5000, A...","[{'label': 'display', 'lat': -34.9208691484817...",-34.920869,138.596527,5000,SA,4d8ad7dad85f3704dc9707dc
8,Platform 5,Platform,Adelaide Train Station,AU,Adelaide,Australia,NaN,462,"[Adelaide Train Station, Adelaide SA 5000, Aus...","[{'label': 'display', 'lat': -34.9212320915168...",-34.921232,138.596671,5000,SA,4df19e3ed4c01ff6b2e47352
9,Platform 9,Platform,Adelaide Railway Station,AU,Adelaide,Australia,NaN,495,"[Adelaide Railway Station, Adelaide SA 5000, A...","[{'label': 'display', 'lat': -34.9207726666211...",-34.920773,138.596359,5000,SA,4cfc991ac51fa1cdfac3ec2b


In [61]:
adelst_df = pd.DataFrame(dataframe_filtered.head(1))
adelst_df

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Adelaide Railway Station,Train Station,138 North Terrace,AU,Adelaide,Australia,NaN,493,"[138 North Terrace, Adelaide SA 5000, Australia]","[{'label': 'display', 'lat': -34.9208454200377...",-34.920845,138.59637,5000,SA,4b440416f964a5203bf025e3


In [62]:
adelstlat = adelst_df.iloc[0]['lat']
adelstlng = adelst_df.iloc[0]['lng']
print('Coordinates successfully loaded!')

Coordinates successfully loaded!


In [63]:
search_query = 'Restaurant'
radius = 500
LIMIT = 50
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, adelstlat, adelstlng, VERSION, search_query, radius, LIMIT)
url

Restaurant .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=JBE44IHG4Y3MP003PZ5ADZCLN5K2LXHCC0YBPF01KIIXBUNL&client_secret=BI5HKYUDKM4GRI45Q0BZSH5FPQCDEMGBB5PCXXUJDLI1BZHQ&ll=-34.92084542003778,138.59637047240594&v=20200531&query=Restaurant&radius=500&limit=50'

In [64]:
results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '52af0bd33cf9994f4e043bdd', 'name': 'H...",False,4c84ac9ee602b1f7210bb17a,21 Hindley Street,AU,North Adelaide,Australia,NaN,291,"[21 Hindley Street, North Adelaide SA 5000, Au...","[{'label': 'display', 'lat': -34.9231411604740...",-34.923141,138.597911,5000,SA,Adelaide Hot Pot Restaurant,v-1590980007,NaN
1,"[{'id': '4bf58dd8d48988d169941735', 'name': 'A...",False,4c9585a358d4b60c18863329,NaN,AU,Adelaide,Australia,NaN,556,"[Adelaide SA, Australia]","[{'label': 'display', 'lat': -34.9165071037749...",-34.916507,138.599407,NaN,SA,Jolly's Boathouse Restaurant,v-1590980007,NaN
2,"[{'id': '4bf58dd8d48988d169941735', 'name': 'A...",False,4d9a8c48b4fa370411c1880d,210 North Tce,AU,Adelaide,Australia,NaN,622,"[210 North Tce, Adelaide SA 5000, Australia]","[{'label': 'display', 'lat': -34.9214960010831...",-34.921496,138.603140,5000,SA,Air Restaurant,v-1590980007,NaN
3,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,514aa10ee4b08933ba007dd5,Intercontinental Adelaide Hotel,AU,Adelaide,Australia,North Terrace,32,[Intercontinental Adelaide Hotel (North Terrac...,"[{'label': 'display', 'lat': -34.9205796196539...",-34.920580,138.596517,5000,SA,Riverside Restaurant,v-1590980007,NaN
4,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",False,4dfb3b12483b085eeda37d6b,Intercontinental Hotel Adelaide,AU,Adelaide,Australia,North Terrace,34,[Intercontinental Hotel Adelaide (North Terrac...,"[{'label': 'display', 'lat': -34.9205636988557...",-34.920564,138.596519,5000,SA,Shiki Japanese restaurant,v-1590980007,NaN


In [65]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Adelaide Hot Pot Restaurant,Hotpot Restaurant,21 Hindley Street,AU,North Adelaide,Australia,NaN,291,"[21 Hindley Street, North Adelaide SA 5000, Au...","[{'label': 'display', 'lat': -34.9231411604740...",-34.923141,138.597911,5000,SA,4c84ac9ee602b1f7210bb17a
1,Jolly's Boathouse Restaurant,Australian Restaurant,NaN,AU,Adelaide,Australia,NaN,556,"[Adelaide SA, Australia]","[{'label': 'display', 'lat': -34.9165071037749...",-34.916507,138.599407,NaN,SA,4c9585a358d4b60c18863329
2,Air Restaurant,Australian Restaurant,210 North Tce,AU,Adelaide,Australia,NaN,622,"[210 North Tce, Adelaide SA 5000, Australia]","[{'label': 'display', 'lat': -34.9214960010831...",-34.921496,138.603140,5000,SA,4d9a8c48b4fa370411c1880d
3,Riverside Restaurant,Restaurant,Intercontinental Adelaide Hotel,AU,Adelaide,Australia,North Terrace,32,[Intercontinental Adelaide Hotel (North Terrac...,"[{'label': 'display', 'lat': -34.9205796196539...",-34.920580,138.596517,5000,SA,514aa10ee4b08933ba007dd5
4,Shiki Japanese restaurant,Japanese Restaurant,Intercontinental Hotel Adelaide,AU,Adelaide,Australia,North Terrace,34,[Intercontinental Hotel Adelaide (North Terrac...,"[{'label': 'display', 'lat': -34.9205636988557...",-34.920564,138.596519,5000,SA,4dfb3b12483b085eeda37d6b
5,Crappy Restaurant At Intercon,None,NaN,AU,NaN,Australia,NaN,89,[Australia],"[{'label': 'display', 'lat': -34.920061, 'lng'...",-34.920061,138.596560,NaN,NaN,4bb1b396f964a520839f3ce3
6,The Balcony Restaurant,Steakhouse,NaN,AU,NaN,Australia,NaN,107,[Australia],"[{'label': 'display', 'lat': -34.9215924912125...",-34.921592,138.597119,NaN,NaN,50f3c6f5e4b0339f7c7f60cb
7,Classics Restaurant,None,NaN,AU,NaN,Australia,NaN,142,[Australia],"[{'label': 'display', 'lat': -34.920043, 'lng'...",-34.920043,138.597592,NaN,NaN,4c21ac4e9085d13aeb2d85cc
8,Pondok Daun Restaurant,Indonesian Restaurant,94 Currie Street,AU,Adelaide,Australia,NaN,411,"[94 Currie Street, Adelaide SA 5000, Australia]","[{'label': 'display', 'lat': -34.9245369748555...",-34.924537,138.596200,5000,SA,54e59bf4498e9da2eb432fe3
9,TG's Restaurant,Buffet,North Tce,AU,Adelaide,Australia,NaN,237,"[North Tce, Adelaide SA 5000, Australia]","[{'label': 'display', 'lat': -34.9217026944170...",-34.921703,138.598757,5000,SA,5077e607e4b0801687c0cede


In [66]:
venues_id = dataframe_filtered['id']
venues_id.head()

0    4c84ac9ee602b1f7210bb17a
1    4c9585a358d4b60c18863329
2    4d9a8c48b4fa370411c1880d
3    514aa10ee4b08933ba007dd5
4    4dfb3b12483b085eeda37d6b
Name: id, dtype: object

In [67]:
column_names=['ID','Name','Rating']
df_adel = pd.DataFrame(get_venue_details(venues_id[0]),columns=column_names)
df_adel

,ID,Name,Rating


In [68]:
# Pass a list of series to the append() to add multiple rows
df_adel = df_adel.append([get_venue_details(venues_id[1]), get_venue_details(venues_id[2]), get_venue_details(venues_id[3]),
                          get_venue_details(venues_id[4]), get_venue_details(venues_id[5]), get_venue_details(venues_id[6]),
                          get_venue_details(venues_id[7]), get_venue_details(venues_id[8]), get_venue_details(venues_id[9]),
                          get_venue_details(venues_id[10]), get_venue_details(venues_id[11]), get_venue_details(venues_id[12]),
                          get_venue_details(venues_id[13]), get_venue_details(venues_id[14]), get_venue_details(venues_id[15]),
                          get_venue_details(venues_id[16])], ignore_index=True)
df_adel

,ID,Name,Rating
0,4c9585a358d4b60c18863329,Jolly's Boathouse Restaurant,8.0
1,4dfb3b12483b085eeda37d6b,Shiki Japanese restaurant,5.7
2,54e59bf4498e9da2eb432fe3,Pondok Daun Restaurant,8.1
3,4c64d5c33f69c9b68cd689d9,Bollywood Indian Restaurant,5.8
4,4b57c976f964a520154128e3,Nando's,7.0
5,4badbca3f964a52026663be3,Cos Leigh St,6.6


In [69]:
df_adel.describe()

,Rating
count,6.000000
mean,6.866667
std,1.038589
min,5.700000
25%,6.000000
50%,6.800000
75%,7.750000
max,8.100000
